<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con redes neuronales convolucionales (CNN)

Ejemplo de clasificación utilizando redes neuronales convolucionales para la clasificación de imagenes<br>

v2.0

### **Objetivos:**
* Estudiar el dataset de los Simpsons.
* Visualizar las imágenes a analizar.
* Normalizar la imágenes de train.
* Construir, entrenar y evaluar dos modelos con Redes Convolucionales.


In [ ]:
#Librerias a implementar
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical # Si esto no funciona, probar con el import anterior

from glob import glob
import matplotlib.image as mpimg

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

### `Simpsons dataset`:
El dataset **`Simpsons`** contiene 550Mbytes de imagenes a color de los personajes de los Simpsons (47 personajes). Cada imagen es de tiene al rededor de 500x450 píxeles a color (3 canales).<br> [Dataset source](https://www.kaggle.com/paultimothymooney/zipfiles)

## Código de descarga del dataset Simpsons

In [ ]:
# Descargar la carpeta imagenes simpsons
if os.access('./simpsons_dataset', os.F_OK) is False:
    !curl -L -o 'simpsons_dataset.zip' 'https://drive.google.com/u/0/uc?id=1mn7cuJ4VudbF1HshbWnvsivSk5o8qAn2&export=download&confirm=t'
    !unzip -q simpsons_dataset.zip
else:
    print("La carpeta ya se encuentra descargada")

In [ ]:
# Visualizar los directiorios o tipos de personas
# os, módulo "sistema operativo"
# .listdir, método que permite listar el contenido dentro de una carpeta
# Dentro de la carpeta simpsons_dataset, cada personaje tiene su propia carpeta de imágenes.
os.listdir("./simpsons_dataset")

In [ ]:
# Se almacena todas las carpetas en la variable "personajes"
personajes = os.listdir("./simpsons_dataset")

# Con la función len() se puede saber la cantidadde carpetas. 
print("Cantidad de tipos de personaejs:", len(personajes))

In [ ]:
# Visualizar las 10 primeras imagenes de un personaje
# glob(), encuentra todos los nombres de rutas que se asemejan a un patrón especificado de acuerdo a las reglas que se siguen.
files = glob("./simpsons_dataset/" + personajes[0] + "/**.jpg")

# plt alias de Matplotlib.
# Método figure() crea el espacio para dibujar.
# Con figsize=(16,9) se define el ancho y alto del dibujo
fig = plt.figure(figsize=(16,9))

# Bucle que itera 10 veces para mostrar las primeras 10 imágenes del dataset
for i in range(10):
    
    # ax gráfico que mostrará las imágenes en 2 filas y 5 columnas
    # En cada iteración va ubicando la imagen en la siguiente posición (i+1)
    ax = fig.add_subplot(2, 5, i+1)
    
    # .axis('off') elimina el recuadro de cada imagen
    ax.axis('off')

    # Herramienta de Matplotlib para para leer imágenes
    img = mpimg.imread(files[i])

    # Muestra las 50 imágenes de la variable data_X_train en el espacio del dibujo
    plt.imshow(img)

# Muestra la figura
plt.show()

In [ ]:
# Visualizar la dimension de la primera imagen
img = mpimg.imread(files[0])
img.shape

In [ ]:
# Visualizar como están representados los pixeles internos.
print(img[85, 100:110, :])

#### Conclusiones
- Las imagenes tienen tamaño variable, utilizaremos un tamaño reducido para que todas las imagenes sean iguales (se elije 150x150)
- Las imagenes están representadas de 0 a 255, hay que normalizarlas

## Código de descarga del dataset Simpsons para test.

In [ ]:
# Descargar datos de test
if os.access('simpsons_test', os.F_OK) is False:
    if os.access('simpsons_test.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://github.com/InoveAlumnos/dataset_analytics_python/raw/master/simpsons_test.zip > simpsons_test.zip
        else:
            !wget simpsons_test.zip https://github.com/InoveAlumnos/dataset_analytics_python/raw/master/simpsons_test.zip
    !unzip -q simpsons_test.zip
else:
    print("El archivo ya se encuentra descargado")

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Se importa ImageDataGenerator del módulo de keras.preprocessing.image
from keras.preprocessing.image import ImageDataGenerator

# Crear un generador, indicando si deseamos realizar un escalado de la imagen
train_datagen = ImageDataGenerator(rescale=1./255)

# El método .flow_from_directory, toma la ruta a un directorio y genera lotes de datos aumentados.
# target_size, se indica la dimensión de la imagen que se desea.
# batch_size, la cantidad que va a tomar para aplicar la operación de escalado.
# class_mode, es categorical ya que son varios personajes.
train_generator = train_datagen.flow_from_directory(
        directory="./simpsons_dataset",
        target_size=(150, 150),
        batch_size=20,
        class_mode="categorical")

# Con dict, arma un diccionario
# con zip, es una función toma que iterables como argumentos y devuelve un iterador.
# Es decir, se construye en diccionario indice:valor --> ubicacion:nombre_personaje
index_to_classes = dict(zip(train_generator.class_indices.values(), train_generator.class_indices.keys()))
index_to_classes

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# El generador "train_generator" se lo puede utilizar para acceder a los datos
# de a cantidad batch de imagenes. En este caso el generador me retornará
# la primera vez las primeras 20 imágenes
# El generador devuelve las imagenes (X) y las clases(personajes) a las que
# pertenece (y)
# X, y = train_generator.next()
batch_imagenes, batch_clases = train_generator.next()

In [ ]:
# Cantidad de imágenes, dimensión alto, dimensión ancho, canales de color
batch_imagenes.shape

In [ ]:
# Cantidad de imágenes categorías (representadas por cada personaje)
batch_clases.shape

In [ ]:
# Cantidad de imágenes.
print("Cantidad de imagenes en el batch:", batch_imagenes.shape[0])

# Dimensión alto, dimensión ancho, canales de color
print("Dimensión de la imagen:", batch_imagenes.shape[1:])

In [ ]:
# batch_clases, variable que contiene la cantidad de personajes.
print("Cantidad de clases/personajes:", batch_clases.shape[1])

In [ ]:
# plt alias de Matplotlib.
# Método figure() crea el espacio para dibujar.
# Con figsize=(16,9) se define el ancho y alto del dibujofig = plt.figure(figsize=(16,9))
# Observar las primeras 5 imagenes de ese batch
fig = plt.figure(figsize=(16,9))

# Itera 5 veces
for i in range(5):

    # ax, gráfico que mostrará las imágenes en 1 filas y 5 columnas
    # En cada iteración va ubicando la imagen en la siguiente posición (i+1)
    ax = fig.add_subplot(1, 5, i+1)

    # Muestra la imagen
    ax.imshow(batch_imagenes[i])

    # Ubica por la posición de la imagen el nombre que le corresponde.
    numero_clase = batch_clases[i].argmax()

    # A cada imagen le agrega un titulo que sería el nombre del personaje que le corresponde.
    ax.set_title(index_to_classes[numero_clase])

# Muestra la imagen.
plt.show()

__Importante__! Luego de usar un generador "jugando", ese batch de imagenes que sacamos ya no se encontrará disponible para ser utilizado en el entrenamiento, es recomendable volver a crear los generadores si se los consumen

In [ ]:
# Crear un generador, indicando para realizar un escalado de la imagen
train_datagen = ImageDataGenerator(rescale=1./255)

# El método .flow_from_directory, toma la ruta a un directorio y genera lotes de datos aumentados.
# target_size, se indica la dimensión de la imagen que se desea.
# batch_size, la cantidad que va a tomar para aplicar la operación de escalado.
# class_mode, es categorical ya que son varios personajes.
train_generator = train_datagen.flow_from_directory(
        directory="./simpsons_dataset",
        target_size=(150, 150),
        batch_size=20,
        class_mode="categorical")

# Con dict, arma un diccionario
# con zip, es una función toma que iterables como argumentos y devuelve un iterador.
# Es decir, se construye en diccionario indice:valor --> ubicacion:nombre_personaje
index_to_classes = dict(zip(train_generator.class_indices.values(), train_generator.class_indices.keys()))
index_to_classes

### Importante: Los generadores ya que encargan de transformar la salida a oneHotEncoding

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

In [ ]:
# input shape (observado del análisis de datos)
# Almacena las dimensiones y los canales de color, sería la entrada a la red
in_shape = (150, 150, 3)
in_shape

In [ ]:
# output shape (observado del análisis de datos)
# 42 ya que representa categorías, los nombres de los personajes con lo que se entrena la red
out_shape = 42
out_shape

### Debemos definir cuantas imagenes se consumiran por epoca (steps_per_epoch)

In [ ]:
# ya que estando el generador en el medio Keras no puede saberlo por
# su cuenta
steps_per_epoch_train = len(train_generator)
steps_per_epoch_train

In [ ]:
# Se importa Dense,  Dropout, Flatten de la librería keras.layers
# Se importa Conv2D, MaxPooling2D  de la librería keras.layers.convolutional
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

# Se crea el objeto model1 a partir de la clase Sequential()
model1 = Sequential()

# Primero realizaremos un modelo muy simple con una solo par de CONV + POOL
# tal cual se utilizo en los otros notebooks de dataset más simples

# .add, método para agregar capas en la red
# Conv2D, agrega una capa convolucional, cuyos parámetros son:
#filters, cantidad de filtros 
# kernel_size,  especifica la altura y el ancho de la ventana de convolución 2D
# strides, especifica las zancadas (de cuánto en cuánto) de la convolución a lo largo de la altura y el ancho.
# MaxPooling2D, achica la imagen.
model1.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=in_shape))
model1.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Capa de comunicación entre la red convolucional y la red neuronal
model1.add(Flatten())

# Red Neuronal
model1.add(Dense(units=64, activation='relu'))

# Capa de salida.
model1.add(Dense(units=out_shape, activation='softmax'))

# Configuración del modelo para el entrenamiento, implementando el método compile a partir del modelo creado.
# Se necesita indicar los parámetros:
# optimizer, nombre del optimizador (es el algoritmo que se encarga del descenso de gradiente estocástico)
# Fuente: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
# loss, se llama función de pérdida, representa las categorías conocidas de las predicción. Al ser 'categorical_crossentropy' 
#la predicción tiene una salida con varias opciones.
# metrics, se define la métrica que evaluará el modelo durante el entrenamiento y las pruebas.
model1.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Resumen de la estructura de la red.
model1.summary()

Se puede observar que esta red tiene más de 2 millones de parámetros para entrenar!!\
Esto es porque la capa densa de POOL de 75x75x8 se transforma a un flatten
de 450000 neuronaes (75x75x8 = 45000) que luego se conectan con todas las 
neuroanes de la capa sigueinte (64) --> 45000x64 + 64 = 2880064\
**NOTA:** Para bajar la cantidad de parametros debemos seguir comprimiendo la imagen


In [ ]:
# Se entrena el modelo con el método fit
# Necesita definir los valores para train_generator, la cantidad de épocas que seria la iteraciones de entrenamiento y
#  steps_per_epoch, cantidad de imágenes a consumir la red por época.
history1 = model1.fit(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=2)

In [ ]:
# Variable epoch_count, que almacena en una lista la cantidad de épocas de train
# history1, es la variable que almacena las predicciones del modelo
# y de ella, se puede acceder a información como su historial (history) del accuracy
epoch_count = range(1, len(history1.history['accuracy']) + 1)

# De Seaborn (sns) se accede al gráfico de línea para representar el 'accuracy'.
sns.lineplot(x=epoch_count,  y=history1.history['accuracy'], label='train')
plt.show()

In [ ]:
# Se importa Dense,  Dropout, Flatten de la librería keras.layers
# Se importa Conv2D, MaxPooling2D  de la librería keras.layers.convolutional
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

# Se crea el objeto model2 a partir de la clase Sequential()
model2 = Sequential()

# Ahora agregaremos más pares de capas CONV + POOL a fin de reducir más la
# dimensión de la imagen antes de llegar a la capa flatten
# Otra estrategia es ir aumentando la cantidad de filtros a medida que crece
# la profundidad de la red

# convolucional f=(3,3), # de filtros: 8, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 8, kernel_size = (3, 3), strides=1, padding='same', activation='relu', input_shape=(150, 150, 3)))
model2.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))
# convolucional f=(3,3), # de filtros: 16, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 16, kernel_size = (3, 3), strides=1, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2, strides=2))
# convolucional f=(3,3), # de filtros: 32, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 32, kernel_size = (3, 3), strides=1, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2, strides=2))
# convolucional f=(3,3), # de filtros: 64, activación relu
# max pooling f=2, s=2
model2.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=1, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2, strides=2))
# capa flatten
model2.add(Flatten())
# capa densa de 64 elementos activación relu
model2.add(Dense(units=128, activation='relu'))
model2.add(Dropout(rate=0.2))
# capa densa con un output de 10 elemento con activación softmax
model2.add(Dense(units=out_shape, activation='softmax'))

# Configuración del modelo para el entrenamiento, implementando el método compile a partir del modelo creado.
# Se necesita indicar los parámetros:
# optimizer, nombre del optimizador (es el algoritmo que se encarga del descenso de gradiente estocástico)
# Fuente: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
# loss, se llama función de pérdida, representa las categorías conocidas de las predicción. Al ser 'categorical_crossentropy' 
#la predicción tiene una salida con varias opciones.
# metrics, se define la métrica que evaluará el modelo durante el entrenamiento y las pruebas.
model2.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Resumen de la estructura de la red.
model2.summary()

In [ ]:
# Se entrena el modelo con el método fit
# Necesita definir los valores para train_generator, la cantidad de épocas que seria la iteraciones de entrenamiento y
#  steps_per_epoch, cantidad de imágenes a consumir la red por época.
history2 = model2.fit(train_generator, steps_per_epoch=steps_per_epoch_train, epochs=2)

In [ ]:
# Variable epoch_count, que almacena en una lista la cantidad de épocas de train
# history2, es la variable que almacena las predicciones del modelo
# y de ella, se puede acceder a información como su historial (history) del accuracy
epoch_count = range(1, len(history2.history['accuracy']) + 1)

# De Seaborn (sns) se accede al gráfico de línea para representar el 'accuracy'.
sns.lineplot(x=epoch_count,  y=history2.history['accuracy'], label='train')
plt.show()

In [ ]:
# Crear un generador, indicando para realizar un escalado de la imagen
test_datagen = ImageDataGenerator(rescale=1./255)

# El método .flow_from_directory, toma la ruta a un directorio y genera lotes de datos aumentados.
# target_size, se indica la dimensión de la imagen que se desea.
# batch_size, la cantidad que va a tomar para aplicar la operación de escalado.
# class_mode, es categorical ya que son varios personajes.
# shuffle, sin desordenar
test_generator = test_datagen.flow_from_directory(
        directory="./simpsons_test",
        target_size=(150, 150),
        batch_size=10,
        class_mode=None,
        shuffle=False)


# Predecir los datos a partir de los datos de test (test_generator)
y_hat_prob = model2.predict(test_generator)

# Resultado de la predicción de la primer imagen.
# Muestra las probabilidades para cada personaje.
# La probabilidad más alta es la predicción.
y_hat_prob[0]

In [ ]:
# Para la probabilidad de la primer imagen, se ubica su ubicación (Pero no tenemos el nombre del personaje)
y_hat = np.argmax(y_hat_prob,axis=1)
y_hat

## ¿Cómo obtener el nombre del personaje de la predicción?

In [ ]:
#¿Cómo obtenemos el "y" verdadero?
# A partir del atributo filanames
test_generator.filenames

#### **Nota:** Los nombres de los personajes de test_generator.filenames tienen barra, extensiones. Por lo que, hay que extraer solo el nombre.
Ejemplo: 


*   De esto --> ['test_images/sideshow_bob_38.jpg']
*   A esto --> [sideshow_bob]




In [ ]:
# Muy rebuscada esta forma de obtener los nombres de los personajes!
# Pero en general cuando tenemos los datos de test no tenemos los nombres
# por lo que no tenemos el "y" verdadero

personajes_test = []

# Bucle que recorre todos los nombres de los personajes de test_generator
# Para extraer sólo el nombre
for file in test_generator.filenames:

    # Ubica la ruta del archivo y alamcena solo el nombre, 
    # por ejemplo: abraham_grampa_simpson_39.jpg
    image_name = os.path.basename(file)
    
    # Una vez ubicado el nombre de la img,
    # separa los elementos por "_", por ejemplo; ['abraham', 'grampa', 'simpson', '39.jpg']
    image_name_split = image_name.split("_")
    
    # Extrae el último elemento que corresponde a la extensión de la imagen,
    # por ejemplo; ['abraham', 'grampa', 'simpson']
    personaje_name_split = image_name_split[:len(image_name_split)-1]
   
    # Nos quedamos con el primer elemento, primer nombre de la lista,
    # por ejemplo; abraham
    personaje = personaje_name_split[0]
  
    # Bucle que recorre la lista con el nombre del personaje
    # desde el primer elemento hasta el final, por ejemplo; ['abraham', 'grampa', 'simpson']
    # Para concatenar el nombre con "_"
    for name in personaje_name_split[1:]:
        personaje += "_" + name
        
    # Agrega el nombre del personaje en una lista 
    personajes_test.append(personaje)
personajes_test

In [ ]:
# Obtener el "y" verdadero
# por cada personaje de la predicción ubica el indice 
# que le corresponde en los datos de train_generator 
y_test = [train_generator.class_indices[personaje] for personaje in personajes_test]
y_test

In [ ]:
# Descargar el modelo entrenado para usar en el futuro sin tener
# que volver a entrenarlo
model2.save("cnn_simpsons.h5")

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat)

In [ ]:
# Se utiliza la matriz de confusión para evaluar la precisión de una clasificación.
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Necesita dos variables que contengan los valores a comparar
cm = confusion_matrix(y_test, y_hat)

# Código para realizar la representación gráfica con los resultados
# Se crea la varible cmd, que almacena visualization de la Confusion Matrix 
# Necesita la variable cm que contiene los resultados de la comparación entre los valores reales y predicción
# display_labels, se especifica las etiquetas de las categorias que se evalúan.
cmd = ConfusionMatrixDisplay(cm, display_labels=list(range(47)))

# Con cmd.plot se especifica el mapa de colores reconocido por matplotlib.
cmd.plot(cmap=plt.cm.Blues)

# Mostrar la figura
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

### Observar los resultados de la predicción.

In [ ]:
#  test_generator.next(), para observar los resultados de las siguientes 10 imágenes de la predección.
batch_test = test_generator.next()

In [ ]:
# plt alias de Matplotlib.
# Método figure() crea el espacio para dibujar.
# Con figsize=(16,9) se define el ancho y alto del dibujofig = plt.figure(figsize=(16,9))
# Observar las primeras 5 imagenes de ese batch
fig = plt.figure(figsize=(16,9))

# Itera 5 veces
for i in range(10):

    # ax, gráfico que mostrará las imágenes en 2 filas y 5 columnas
    # En cada iteración va ubicando la imagen en la siguiente posición (i+1)
    ax = fig.add_subplot(2, 5, i+1)

    # Muestra la imagen
    ax.imshow(batch_test[i])

    # Ubica por la posición de la imagen el nombre que le corresponde.
    numero_clase = y_hat[i]

    # A cada imagen le agrega un titulo que sería el nombre del personaje que le corresponde.
    ax.set_title(index_to_classes[numero_clase])

# Muestra la imagen.
plt.show()

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

Al utilizar más pares de capas CONV+POOL se pudo obtener un mejor resultado, un modelo casi perfecto. Hay que tener en cuenta que el dataset de test es muy pequeño y hay muchos otros personajes que no estamos clasificando.